Submitted by:
1. Natan Izhak Poor 211412986
2. Liraz Lizi Cohen 318865854
3. Khaled Khalaf 207966607
Submission date: 12/08/2024

Install necessary libraries

In [1]:
pip install openpyxl pandas networkx numpy

Note: you may need to restart the kernel to use updated packages.


Import necessary libraries

In [14]:
import pandas as pd
import requests
import networkx as nx
import numpy as np

Read the excel dataset to a dataframe named df from GitHub

In [ ]:
df = pd.read_excel('https://raw.githubusercontent.com/NatiIPReact/DivvyTripsData/main/PreProcessing/Divvy_Trips.xlsx')

If getting the data from GitHub doesn't work for you, you can use:

!git clone https://github.com/NatiIPReact/DivvyTripsData
dfQ1 = pd.read_excel('DivvyTripsData/PreProcessing/Divvy_Trips.xlsx')

Instead of the cell above

Create a dictionary of station as keys and its coordinates as values

In [3]:
# Initialize an empty dictionary to store the unique places
places_dict = {}

# Function to add place to the dictionary
def add_place(place, lat, lon):
    if place not in places_dict:
        places_dict[place] = (lat, lon)

# Iterate over the rows of the DataFrame and populate the dictionary
for idx, row in df.iterrows():
    add_place(row['source'], row['FROM LATITUDE'], row['FROM LONGITUDE'])
    add_place(row['target'], row['TO LATITUDE'], row['TO LONGITUDE'])

Using Google's API, get the neighborhood of each station using the coordinates, we'll store those in a unique Neighborhoods list, and then find the median income manually to add some spicy data.

In [4]:
GOOGLE_API_KEY1 = 'AIzaSyDFOFJdjn7EibQ'
GOOGLE_API_KEY2 = 'WPe2lGiw2TC47RfYMsJ8'
GOOGLE_API_KEY = GOOGLE_API_KEY1 + GOOGLE_API_KEY2
def reverse_geocode(lat, lng):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={GOOGLE_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        results = response.json()['results']
        for result in results:
            if 'neighborhood' in result['types']:
                return result['address_components'][0]['long_name']
    return None

results = []
Neighborhoods=[]
for station, (lat, lng) in places_dict.items():
    neighborhood = reverse_geocode(lat, lng)
    if neighborhood not in Neighborhoods:
        Neighborhoods.append(neighborhood)
    results.append({
        'Station': station,
        'Latitude': lat,
        'Longitude': lng,
        'Neighborhood': neighborhood
    })

Let's print the results so we can see it's good

In [5]:
results

[{'Station': 'Michigan Ave & Oak St',
  'Latitude': 41.90096039,
  'Longitude': -87.62377664,
  'Neighborhood': 'Gold Coast'},
 {'Station': 'Millennium Park',
  'Latitude': 41.8810317,
  'Longitude': -87.62408432,
  'Neighborhood': 'Millennium Park'},
 {'Station': 'Cannon Dr & Fullerton Ave',
  'Latitude': 41.92675599,
  'Longitude': -87.63442879,
  'Neighborhood': 'Lincoln Park'},
 {'Station': 'Lake Shore Dr & Monroe St',
  'Latitude': 41.880958,
  'Longitude': -87.616743,
  'Neighborhood': 'Chicago Loop'},
 {'Station': 'Ashland Ave & Armitage Ave',
  'Latitude': 41.916017,
  'Longitude': -87.668879,
  'Neighborhood': 'Bucktown'},
 {'Station': 'Dayton St & North Ave',
  'Latitude': 41.91057804,
  'Longitude': -87.64942193,
  'Neighborhood': 'Goose Island'},
 {'Station': 'McClurg Ct & Illinois St',
  'Latitude': 41.89102,
  'Longitude': -87.6173,
  'Neighborhood': 'Streeterville'},
 {'Station': 'Fairbanks Ct & Grand Ave',
  'Latitude': 41.89186,
  'Longitude': -87.62062,
  'Neighborhoo

Let's create a dict mapping a station to its area in Chicago

In [6]:
station_to_neighborhood = {result['Station']: result['Neighborhood'] for result in results}
station_to_neighborhood

{'Michigan Ave & Oak St': 'Gold Coast',
 'Millennium Park': 'Millennium Park',
 'Cannon Dr & Fullerton Ave': 'Lincoln Park',
 'Lake Shore Dr & Monroe St': 'Chicago Loop',
 'Ashland Ave & Armitage Ave': 'Bucktown',
 'Dayton St & North Ave': 'Goose Island',
 'McClurg Ct & Illinois St': 'Streeterville',
 'Fairbanks Ct & Grand Ave': 'Streeterville',
 'McCormick Place': 'South Loop',
 'State St & Kinzie St': 'Near North',
 'Dearborn St & Adams St': 'Chicago Loop',
 'Franklin St & Lake St': 'Chicago Loop',
 'California Ave & 21st St': 'Marshall Square',
 'State St & Van Buren St': 'Chicago Loop',
 'Lincoln Ave & Armitage Ave': 'Old Town Triangle',
 'Larrabee St & Menomonee St': 'Old Town',
 'Michigan Ave & Lake St': 'New Eastside',
 'Dearborn St & Monroe St': 'Chicago Loop',
 'Southport Ave & Wrightwood Ave': 'Wrightwood Neighbors',
 'Sheffield Ave & Willow St': 'Ranch Triangle',
 'Clark St & Wrightwood Ave': 'Park West',
 'Orleans St & Elm St': 'Near North Side',
 'May St & Randolph St': 'W

In [7]:
NeighborhoodsIncome = {
 'Gold Coast': 108295,
 'Millennium Park': 123115,
 'Lincoln Park': 123610,
 'Chicago Loop': 123115,
 'Bucktown': 117152,
 'Goose Island': 115224,
 'Streeterville': 117344,
 'South Loop': 109634,
 'Near North': 52011,
 'Marshall Square': 48791,
 'Old Town Triangle': 123610,
 'Old Town': 99246,
 'New Eastside': 46156,
 'Wrightwood Neighbors': 55442,
 'Ranch Triangle': 123610,
 'Park West': 92335,
 'Near North Side': 117344,
 'West Loop': 130196,
 'Wicker Park': 110694,
 'Fulton Market': 124348,
 'Little Italy': 85562,
 'West Loop Gate': 130196,
 'Noble Square': 131201,
 'River North': 127072,
 'Pilsen': 61490,
 'West Town': 104693,
 'Bridgeport': 47778,
 'Ravenswood': 69874,
 'Cabrini-Green': 99246,
 'Uptown': 63660,
 'Near West Side': 80880,
 'Fulton River District': 133322,
 'East Village': 110694,
 'Grand Boulevard': 39111,
 "Printer's Row": 112162,
 'Museum Campus': 115993,
 'Wrigleyville': 93362,
 'North Kenwood': 34564,
 'Lake View East': 93362,
 'River West': 129136,
 'Hyde Park': 52423,
 'Kenwood': 52336,
 'Grant Park': 76750,
 'Lake View': 95173,
 'Buena Park': 88267,
 'North Center': 89958,
 'The Gap': 65489,
 'Douglas': 35796,
 'West Lakeview': 110496,
 'Heart of Chicago': 61490,
 'Near South Side': 115993,
 'West DePaul': 123610,
 'Palmer Square': 89974,
 'Ukrainian Village': 110694,
 'East Pilsen': 63490,
 'Illinois Medical District': 18100,
 'Rush & Division': 84102,
 'Mid-North District': 123004,
 'Chinatown': 65489,
 'Washington Park': 33036,
 'Prairie District': 116027,
 'Sheffield Neighbors': 204211,
 'Northalsted': 61002,
 'Logan Square': 84653,
 'Roscoe Village': 95054,
 'East Hyde Park': 41450,
 'Lake Meadows': 65489,
 'Prairie Shores': 65489,
 'South Commons': 65489,
 'Pulaski Park': 57007,
 'Edgewater': 55768,
 'Bronzeville': 43048,
 'Dearborn Homes': 65489,
 'Heart of Italy': 61490,
 'Fulton Market District': 129624
}

Now we add "SourceArea" and "TargetArea" columns to the dataframe

In [8]:
df['SourceArea'] = df['source'].map(station_to_neighborhood)
df['TargetArea'] = df['target'].map(station_to_neighborhood)
df

,TRIP ID,START TIME,STOP TIME,BIKE ID,TRIP DURATION,FROM STATION ID,source,TO STATION ID,target,USER TYPE,GENDER,BIRTH YEAR,FROM LATITUDE,FROM LONGITUDE,FROM LOCATION,TO LATITUDE,TO LONGITUDE,TO LOCATION,SourceArea,TargetArea
0,10000,6/30/2013 10:46,6/30/2013 11:41,338,3260,85,Michigan Ave & Oak St,90,Millennium Park,Customer,NaN,NaN,41.900960,-87.623777,POINT (-87.623777 41.90096),41.881032,-87.624084,POINT (-87.624084 41.881032),Gold Coast,Millennium Park
1,10004,6/30/2013 10:48,6/30/2013 11:15,345,1664,34,Cannon Dr & Fullerton Ave,90,Millennium Park,Subscriber,Female,1986.0,41.926756,-87.634429,POINT (-87.634429 41.926756),41.881032,-87.624084,POINT (-87.624084 41.881032),Lincoln Park,Millennium Park
2,10005,6/30/2013 10:48,6/30/2013 11:13,425,1521,76,Lake Shore Dr & Monroe St,90,Millennium Park,Customer,NaN,NaN,41.880958,-87.616743,POINT (-87.616743 41.880958),41.881032,-87.624084,POINT (-87.624084 41.881032),Chicago Loop,Millennium Park
3,10006,6/30/2013 10:48,6/30/2013 11:18,580,1793,34,Cannon Dr & Fullerton Ave,76,Lake Shore Dr & Monroe St,Customer,NaN,NaN,41.926756,-87.634429,POINT (-87.634429 41.926756),41.880958,-87.616743,POINT (-87.616743 41.880958),Lincoln Park,Chicago Loop
4,10008,6/30/2013 10:48,6/30/2013 11:13,448,1509,76,Lake Shore Dr & Monroe St,90,Millennium Park,Customer,NaN,NaN,41.880958,-87.616743,POINT (-87.616743 41.880958),41.881032,-87.624084,POINT (-87.624084 41.881032),Chicago Loop,Millennium Park
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652624,2007267,6/7/2014 16:15,6/7/2014 16:40,2794,1513,35,Streeter Dr & Illinois St,143,Sedgwick St & Webster Ave,Customer,NaN,NaN,41.891071,-87.612200,POINT (-87.6122 41.891071),41.922167,-87.638888,POINT (-87.638888 41.922167),Streeterville,Mid-North District
652625,2007268,6/7/2014 16:15,6/7/2014 16:44,670,1740,177,Theater on the Lake,219,Damen Ave & Cortland St,Customer,NaN,NaN,41.926277,-87.630834,POINT (-87.630834 41.926277),41.916027,-87.677411,POINT (-87.677411 41.916027),Lincoln Park,Bucktown
652626,2007269,6/7/2014 16:15,6/7/2014 16:20,1194,288,44,State St & Randolph St,44,State St & Randolph St,Subscriber,Male,1968.0,41.884730,-87.627734,POINT (-87.627734 41.88473),41.884730,-87.627734,POINT (-87.627734 41.88473),Chicago Loop,Chicago Loop
652627,2007270,6/7/2014 16:15,6/7/2014 16:33,466,1094,94,Clark St & Armitage Ave,327,Sheffield Ave & Webster Ave,Subscriber,Female,1992.0,41.918306,-87.636282,POINT (-87.636282 41.918306),41.921540,-87.653818,POINT (-87.653818 41.92154),Old Town Triangle,Sheffield Neighbors


Now using the added columns, add the corresponding median income of these area

In [9]:
df['SourceMedianIncome'] = df['SourceArea'].map(NeighborhoodsIncome)
df['TargetMedianIncome'] = df['TargetArea'].map(NeighborhoodsIncome)
df

,TRIP ID,START TIME,STOP TIME,BIKE ID,TRIP DURATION,FROM STATION ID,source,TO STATION ID,target,USER TYPE,...,FROM LATITUDE,FROM LONGITUDE,FROM LOCATION,TO LATITUDE,TO LONGITUDE,TO LOCATION,SourceArea,TargetArea,SourceMedianIncome,TargetMedianIncome
0,10000,6/30/2013 10:46,6/30/2013 11:41,338,3260,85,Michigan Ave & Oak St,90,Millennium Park,Customer,...,41.900960,-87.623777,POINT (-87.623777 41.90096),41.881032,-87.624084,POINT (-87.624084 41.881032),Gold Coast,Millennium Park,108295,123115
1,10004,6/30/2013 10:48,6/30/2013 11:15,345,1664,34,Cannon Dr & Fullerton Ave,90,Millennium Park,Subscriber,...,41.926756,-87.634429,POINT (-87.634429 41.926756),41.881032,-87.624084,POINT (-87.624084 41.881032),Lincoln Park,Millennium Park,123610,123115
2,10005,6/30/2013 10:48,6/30/2013 11:13,425,1521,76,Lake Shore Dr & Monroe St,90,Millennium Park,Customer,...,41.880958,-87.616743,POINT (-87.616743 41.880958),41.881032,-87.624084,POINT (-87.624084 41.881032),Chicago Loop,Millennium Park,123115,123115
3,10006,6/30/2013 10:48,6/30/2013 11:18,580,1793,34,Cannon Dr & Fullerton Ave,76,Lake Shore Dr & Monroe St,Customer,...,41.926756,-87.634429,POINT (-87.634429 41.926756),41.880958,-87.616743,POINT (-87.616743 41.880958),Lincoln Park,Chicago Loop,123610,123115
4,10008,6/30/2013 10:48,6/30/2013 11:13,448,1509,76,Lake Shore Dr & Monroe St,90,Millennium Park,Customer,...,41.880958,-87.616743,POINT (-87.616743 41.880958),41.881032,-87.624084,POINT (-87.624084 41.881032),Chicago Loop,Millennium Park,123115,123115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652624,2007267,6/7/2014 16:15,6/7/2014 16:40,2794,1513,35,Streeter Dr & Illinois St,143,Sedgwick St & Webster Ave,Customer,...,41.891071,-87.612200,POINT (-87.6122 41.891071),41.922167,-87.638888,POINT (-87.638888 41.922167),Streeterville,Mid-North District,117344,123004
652625,2007268,6/7/2014 16:15,6/7/2014 16:44,670,1740,177,Theater on the Lake,219,Damen Ave & Cortland St,Customer,...,41.926277,-87.630834,POINT (-87.630834 41.926277),41.916027,-87.677411,POINT (-87.677411 41.916027),Lincoln Park,Bucktown,123610,117152
652626,2007269,6/7/2014 16:15,6/7/2014 16:20,1194,288,44,State St & Randolph St,44,State St & Randolph St,Subscriber,...,41.884730,-87.627734,POINT (-87.627734 41.88473),41.884730,-87.627734,POINT (-87.627734 41.88473),Chicago Loop,Chicago Loop,123115,123115
652627,2007270,6/7/2014 16:15,6/7/2014 16:33,466,1094,94,Clark St & Armitage Ave,327,Sheffield Ave & Webster Ave,Subscriber,...,41.918306,-87.636282,POINT (-87.636282 41.918306),41.921540,-87.653818,POINT (-87.653818 41.92154),Old Town Triangle,Sheffield Neighbors,123610,204211


Now the financial data is ready!
Let's export it to an excel file to backup before continuing

In [10]:
df.to_excel('DivvyTripsWithIncome.xlsx', index=False)

Now, let's add the SourceProximityIndex and TargetProximityIndex column which contains data on how close the station is to important places around Chicago.
First, let's load the data again to start fresh.

In [11]:
df = pd.read_excel('DivvyTripsWithIncome.xlsx')

Build a list of objects contaning the station and its location (latitude & longitude)

In [13]:
places_dict = {}
def add_place(place, lat, lon):
    if place not in places_dict:
        places_dict[place] = (lat, lon)
for idx, row in df.iterrows():
    add_place(row['source'], row['FROM LATITUDE'], row['FROM LONGITUDE'])
    add_place(row['target'], row['TO LATITUDE'], row['TO LONGITUDE'])
results = []
for station, (lat, lng) in places_dict.items():
    results.append({
        'Station': station,
        'Latitude': lat,
        'Longitude': lng
    })
results

[{'Station': 'Michigan Ave & Oak St',
  'Latitude': 41.90096039,
  'Longitude': -87.62377664},
 {'Station': 'Millennium Park',
  'Latitude': 41.8810317,
  'Longitude': -87.62408432},
 {'Station': 'Cannon Dr & Fullerton Ave',
  'Latitude': 41.92675599,
  'Longitude': -87.63442879},
 {'Station': 'Lake Shore Dr & Monroe St',
  'Latitude': 41.880958,
  'Longitude': -87.616743},
 {'Station': 'Ashland Ave & Armitage Ave',
  'Latitude': 41.916017,
  'Longitude': -87.668879},
 {'Station': 'Dayton St & North Ave',
  'Latitude': 41.91057804,
  'Longitude': -87.64942193},
 {'Station': 'McClurg Ct & Illinois St',
  'Latitude': 41.89102,
  'Longitude': -87.6173},
 {'Station': 'Fairbanks Ct & Grand Ave',
  'Latitude': 41.89186,
  'Longitude': -87.62062},
 {'Station': 'McCormick Place',
  'Latitude': 41.85137517,
  'Longitude': -87.61883465},
 {'Station': 'State St & Kinzie St',
  'Latitude': 41.88918,
  'Longitude': -87.6277},
 {'Station': 'Dearborn St & Adams St',
  'Latitude': 41.87935636,
  'Long

Now let's calculate the distance of each station to the important sites of the city. It's being added to a temporary results value. Then we calcualte the actual ProximityIndex value which is a mean of all distances to important sites in the city.
Note: We've chosen the important sites of Chicago to be: Millennium Park, Union Station, City Central, The Art Institute of Chicago

In [16]:
# Function to calculate the Haversine distance between two points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# List of important sites with their coordinates
important_sites = {
    'Millennium Park': (41.8826, -87.6233),
    'Union Station': (41.8786, -87.6406),
    'City Central': (41.8803, -87.6298),
    'The Art Institute of Chicago': (41.8796, 87.6224)
}
# Initialize the dictionary to store closeness values
closeness_dict = {station['Station']: {} for station in results}
# Calculate the distance from each important site for each station and normalize
for site_name, (site_lat, site_lon) in important_sites.items():
    # Calculate the distance from the site for each station
    for station in results:
        distance = haversine(site_lat, site_lon, station['Latitude'], station['Longitude'])
        station[f'DistanceTo{site_name.replace(" ", "")}'] = distance
    # Get the maximum distance to normalize the distances
    max_distance = max(station[f'DistanceTo{site_name.replace(" ", "")}'] for station in results)
    # Normalize the distances to a 0-1 scale and calculate closeness
    for station in results:
        normalized_distance = station[f'DistanceTo{site_name.replace(" ", "")}'] / max_distance
        station[f'ClosenessTo{site_name.replace(" ", "")}'] = 1 - normalized_distance
        closeness_dict[station['Station']][site_name] = 1 - normalized_distance

# Convert the list to a DataFrame for better visualization
dfResults = pd.DataFrame(results)
# Calculate the maximum closeness to any site for each station
closeness_columns = [f'ClosenessTo{site.replace(" ", "")}' for site in important_sites.keys()]
dfResults['MaxClosenessToAnySite'] = dfResults[closeness_columns].max(axis=1)
dfResults

,Station,Latitude,Longitude,DistanceToMillenniumPark,ClosenessToMillenniumPark,DistanceToUnionStation,ClosenessToUnionStation,DistanceToCityCentral,ClosenessToCityCentral,DistanceToTheArtInstituteofChicago,ClosenessToTheArtInstituteofChicago,MaxClosenessToAnySite
0,Michigan Ave & Oak St,41.900960,-87.623777,2.041963,0.815475,2.849788,0.745660,2.350812,0.789750,10686.897515,0.001153,0.815475
1,Millennium Park,41.881032,-87.624084,0.186083,0.983184,1.393809,0.875604,0.480138,0.957058,10689.110794,0.000946,0.983184
2,Cannon Dr & Fullerton Ave,41.926756,-87.634429,4.995554,0.548569,5.379003,0.519930,5.179855,0.536728,10684.089327,0.001416,0.548569
3,Lake Shore Dr & Monroe St,41.880958,-87.616743,0.572719,0.948245,1.992445,0.822177,1.083446,0.903100,10689.081228,0.000949,0.948245
4,Ashland Ave & Armitage Ave,41.916017,-87.668879,5.295056,0.521504,4.773752,0.573948,5.121976,0.541904,10685.457011,0.001288,0.573948
...,...,...,...,...,...,...,...,...,...,...,...,...
301,Wells St & Hubbard St,41.889933,-87.634262,1.219972,0.889755,1.365038,0.878172,1.133041,0.898664,10688.175145,0.001034,0.898664
302,Damen Ave & Cortland St,41.916027,-87.677411,5.819979,0.474069,5.157743,0.539677,5.595548,0.499549,10685.499256,0.001284,0.539677
303,Ogden Ave & Congress Pkwy,41.875010,-87.673280,4.223092,0.618374,2.734988,0.755905,3.647565,0.673772,10690.030547,0.000860,0.755905
304,Marshfield Ave & Cortland St,41.916017,-87.668879,5.295056,0.521504,4.773752,0.573948,5.121976,0.541904,10685.457011,0.001288,0.573948


Now let's get a dictionary of station: ProximityIndex

In [17]:
closeness_dict = dfResults.set_index('Station')['MaxClosenessToAnySite'].to_dict()
closeness_dict

{'Michigan Ave & Oak St': 0.8154748917837064,
 'Millennium Park': 0.9831843121927347,
 'Cannon Dr & Fullerton Ave': 0.5485692050818374,
 'Lake Shore Dr & Monroe St': 0.9482453594222695,
 'Ashland Ave & Armitage Ave': 0.5739480389241697,
 'Dayton St & North Ave': 0.6760269651381137,
 'McClurg Ct & Illinois St': 0.904225107882118,
 'Fairbanks Ct & Grand Ave': 0.9048175107453951,
 'McCormick Place': 0.7010995285542816,
 'State St & Kinzie St': 0.9261422286116768,
 'Dearborn St & Adams St': 0.9906152927392776,
 'Franklin St & Lake St': 0.9306218933184025,
 'California Ave & 21st St': 0.5269095288664768,
 'State St & Van Buren St': 0.9657668437032434,
 'Lincoln Ave & Armitage Ave': 0.6283102335097672,
 'Larrabee St & Menomonee St': 0.6445715129990961,
 'Michigan Ave & Lake St': 0.9650559759617234,
 'Dearborn St & Monroe St': 0.989649547362255,
 'Southport Ave & Wrightwood Ave': 0.4739093585533576,
 'Sheffield Ave & Willow St': 0.6402107451357931,
 'Clark St & Wrightwood Ave': 0.505538950234

Now let's add the ProximityIndex value to our actual dataset to all trips (For both source and target stations)

In [18]:
df['SourceProximityIndex'] = df['source'].map(closeness_dict)
df['TargetProximityIndex'] = df['target'].map(closeness_dict)
df

,TRIP ID,START TIME,STOP TIME,BIKE ID,TRIP DURATION,FROM STATION ID,source,TO STATION ID,target,USER TYPE,...,FROM LOCATION,TO LATITUDE,TO LONGITUDE,TO LOCATION,SourceArea,TargetArea,SourceMedianIncome,TargetMedianIncome,SourceProximityIndex,TargetProximityIndex
0,10000,6/30/2013 10:46,6/30/2013 11:41,338,3260,85,Michigan Ave & Oak St,90,Millennium Park,Customer,...,POINT (-87.623777 41.90096),41.881032,-87.624084,POINT (-87.624084 41.881032),Gold Coast,Millennium Park,108295,123115,0.815475,0.983184
1,10004,6/30/2013 10:48,6/30/2013 11:15,345,1664,34,Cannon Dr & Fullerton Ave,90,Millennium Park,Subscriber,...,POINT (-87.634429 41.926756),41.881032,-87.624084,POINT (-87.624084 41.881032),Lincoln Park,Millennium Park,123610,123115,0.548569,0.983184
2,10005,6/30/2013 10:48,6/30/2013 11:13,425,1521,76,Lake Shore Dr & Monroe St,90,Millennium Park,Customer,...,POINT (-87.616743 41.880958),41.881032,-87.624084,POINT (-87.624084 41.881032),Chicago Loop,Millennium Park,123115,123115,0.948245,0.983184
3,10006,6/30/2013 10:48,6/30/2013 11:18,580,1793,34,Cannon Dr & Fullerton Ave,76,Lake Shore Dr & Monroe St,Customer,...,POINT (-87.634429 41.926756),41.880958,-87.616743,POINT (-87.616743 41.880958),Lincoln Park,Chicago Loop,123610,123115,0.548569,0.948245
4,10008,6/30/2013 10:48,6/30/2013 11:13,448,1509,76,Lake Shore Dr & Monroe St,90,Millennium Park,Customer,...,POINT (-87.616743 41.880958),41.881032,-87.624084,POINT (-87.624084 41.881032),Chicago Loop,Millennium Park,123115,123115,0.948245,0.983184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652624,2007267,6/7/2014 16:15,6/7/2014 16:40,2794,1513,35,Streeter Dr & Illinois St,143,Sedgwick St & Webster Ave,Customer,...,POINT (-87.6122 41.891071),41.922167,-87.638888,POINT (-87.638888 41.922167),Streeterville,Mid-North District,117344,123004,0.881088,0.585680
652625,2007268,6/7/2014 16:15,6/7/2014 16:44,670,1740,177,Theater on the Lake,219,Damen Ave & Cortland St,Customer,...,POINT (-87.630834 41.926277),41.916027,-87.677411,POINT (-87.677411 41.916027),Lincoln Park,Bucktown,123610,117152,0.557519,0.539677
652626,2007269,6/7/2014 16:15,6/7/2014 16:20,1194,288,44,State St & Randolph St,44,State St & Randolph St,Subscriber,...,POINT (-87.627734 41.88473),41.884730,-87.627734,POINT (-87.627734 41.88473),Chicago Loop,Chicago Loop,123115,123115,0.960525,0.960525
652627,2007270,6/7/2014 16:15,6/7/2014 16:33,466,1094,94,Clark St & Armitage Ave,327,Sheffield Ave & Webster Ave,Subscriber,...,POINT (-87.636282 41.918306),41.921540,-87.653818,POINT (-87.653818 41.92154),Old Town Triangle,Sheffield Neighbors,123610,204211,0.628310,0.562821


Now our dataset is finished and ready for the research project!
Let's export it to an excel file and start working!

In [19]:
df.to_excel('DivvyTripsReady.xlsx', index=False)